In [57]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import urllib
import os
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import matplotlib

### Inside the quotations in the box below, input the directory where you have saved the "SuwData" and this script file.

In [58]:
directory = "C:/Users/robert.taylor/Documents/"

### List sites below

In [59]:
searchfor = ['02320700', '02321000', '02321500', '02321898', '02321900', '02321958',
 '02321975', '02322016', '02322049', '02322050', '02322500', '02322616',
 '02322700', '02322703', '02322800']

### The box below pulls the data using a For-loop where it places each site number into the url, reads the html table, and stores the data in a Python dictionary. The dictionary is accessed using "keys" which are automatically stored as "site_no_xxxxxxxx".

In [60]:
count = 0
usgs_dict={}


for i in range(len(usgs_sites_retrieve)):
    usgs_site_no = usgs_sites_retrieve[i]
    url = 'https://waterdata.usgs.gov/nwis/dv?referred_module=sw&search_site_no='+usgs_site_no+'&search_site_no_match_type=exact&site_tp_cd=OC&site_tp_cd=OC-CO&site_tp_cd=ES&site_tp_cd=LK&site_tp_cd=ST&site_tp_cd=ST-CA&site_tp_cd=ST-DCH&site_tp_cd=ST-TS&index_pmcode_00060=1&index_pmcode_00065=1&group_key=NONE&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&range_selection=date_range&begin_date=1980-01-01&end_date=2022-01-20&format=rdb&date_format=YYYY-MM-DD&rdb_compression=value&list_of_search_criteria=search_site_no%2Csite_tp_cd%2Crealtime_parameter_selection'
    page = urllib.request.urlopen(url)
    skiplines = str(page.read())
    skiprow = skiplines.count('#')
    usgs_site_data = pd.read_csv(url,sep='\t',skiprows=skiprow)
    usgs_site_data.drop([0],axis=0,inplace=True)
    
            
            
    
    usgs_dict['site_no_'+str(usgs_site_no)] = usgs_site_data
    count += 1


### Due to the USGS system of reporting data that I am referencing, the column headers were abstract and problematic. Each parameter has a unique identifier, there is a different identifier if it is mena, max, or min, and there is a separate column for reporting data quality / status. To combat this I used regex to extract the relevant information and title the columns automatically. This also must be performed in a loop.

In [61]:
parm_codes=pd.read_csv(directory + 'SuwData/USGS_All_Parameter_Codes.csv', converters={'site_no':lambda x: str(x)})


In [62]:
'''header = usgs_dict['site_no_'+usgs_site_no+'_2001'].columns.values.tolist()
header = pd.DataFrame(header,columns=['cols'])
header = header.cols.str.extract(r'^\w\w\w\w\w_([^_]+)_(.+)',expand=False)
parm_codes=pd.read_csv('C:/Users/Robert.Taylor//pythoncode//parameter_cd_query.txt', skiprows=7, converters={'parm_cd':lambda x: str(x)})
'''
keys = list(usgs_dict)
print('Keys: ' + str(keys))

hdr = {}

for x in range(len(usgs_dict)):
    
    header = usgs_dict[keys[x]].columns.values.tolist()
    header = pd.DataFrame(header,columns=['cols'])
    header = header.cols.str.extract(r'^\w{3,8}_([^_]+)_(.+)',expand=False)
    
    for i in range(header.shape[0]):
        for j in range(header.shape[1]):
            if header.iloc[i,j] == '00001':
                header.iloc[i,j] = ', Max value'
            elif header.iloc[i,j] == '00002':
                header.iloc[i,j] = ', Min value'
            elif header.iloc[i,j] == '00003':
                header.iloc[i,j] = ', Mean value'
            elif header.iloc[i,j] == '00001_cd':
                header.iloc[i,j] = ' Data status / quality'
            elif header.iloc[i,j] == '00002_cd':
                header.iloc[i,j] = ' Data status / quality'
            elif header.iloc[i,j] == '00003_cd':
                header.iloc[i,j] = ' Data status / quality'
            elif pd.isna(header.iloc[i,j]) is False:
                header.iloc[i,j] = parm_codes.loc[parm_codes['parm_cd'] == str(header.iloc[i,j])]['parm_desc'].values
    
    hdr[keys[x]] = header
        

for i in range(len(usgs_dict)):
    hdr[keys[i]]['header'] = hdr[keys[i]][0].astype(str) + hdr[keys[i]][1].astype(str)
    hdr[keys[i]] = hdr[keys[i]]['header']
    hdr[keys[i]].iloc[0] = 'agency_cd'
    hdr[keys[i]].iloc[1] = 'site_no'
    hdr[keys[i]].iloc[2] = 'datetime'
    

#display(hdr[keys[0]].values.tolist())



header_usgs = {}



for i in range(len(usgs_dict)):
    header_usgs[keys[i]] = usgs_dict[keys[i]].columns.values.tolist()
    usgs_dict[keys[i]].columns = hdr[keys[i]].values 
    


Keys: ['site_no_02320700', 'site_no_02321000', 'site_no_02321500', 'site_no_02321898', 'site_no_02321900', 'site_no_02321958', 'site_no_02321975', 'site_no_02322016', 'site_no_02322049', 'site_no_02322050', 'site_no_02322500', 'site_no_02322616', 'site_no_02322700', 'site_no_02322703', 'site_no_02322800']


### Shown below is an example of the replaced column headers and associated data.

In [67]:
usgs_meta_df = pd.read_csv(directory+'/SuwData/USGS_All_Sites_In_Basin.csv')
#site_nm_df = usgs_meta_df.loc[usgs_meta_df['site_no']==searchfor]
#site_nm_df = np.where(usgs_meta_df['site_no']==searchfor)
site_nm_df = usgs_meta_df.loc[usgs_meta_df['site_no'].isin(searchfor)]
site_nm_df
#usgs_meta_df['site_no']
#avail_parm_codes_stream = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
#avail_parms = usgs_parm_codes_df.loc[usgs_parm_codes_df['parm_cd'] == num_parms]
#site_nm_df

,Unnamed: 0,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu


### The following box saves each site's data as a csv in the previously designated folder.

In [68]:
for i in range(len(usgs_dict)):
    #usgs_dict[keys[i]] = id_search_results_usgs.merge(usgs_dict[keys[i]],on='site_no')
    #usgs_dict[keys[i]] = usgs_dict[keys[i]].merge(id_search_results_usgs,on='site_no')
    usgs_dict[keys[i]].to_csv(directory+'/SuwData//USGS_'+keys[i]+'_data.csv')